<a href="https://colab.research.google.com/github/mmcinnestaylor/NMA-CN-2022/blob/main/project/steinmetz_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os, csv, math, torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from datetime import datetime
from sklearn.model_selection import train_test_split
from torch import nn
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [ ]:
torch.set_default_dtype(torch.float64) 

In [5]:
BASE_PATH = '/content/drive/MyDrive/steinmetz' # Path to dataset base directory

In [6]:
# @title Helper Functions
#@markdown - get_spike_trains(clustered_spikes, start_time=0, bin_size=10, window_size=100, format='binary')
#@markdown  - Generates spike trains for each valid cluster in a recording session.
#@markdown - get_area_spike_trains(clustered_spikes, clusters_locs, start_time=0, bin_size=10, window_size=100, format='counts', aggregate=True)
#@markdown  - Generates spike trains separated by brain area
#@markdown - clusters_to_area(cluster_ids, cluster_locs)
#@markdown  - Generates a dictionary keyed by brain areas monitored during a recording session. The values are lists of np arrays corresponding the the cluster
#@markdown - group_trial_timestamps(intervals, vis_stims, go_cues, responses, feedback)
#@markdown  - Groups all timepoints per trial: start, visual stimulus, go cue, response, feedback, trial end

def get_spike_trains(clustered_spikes, start_time=0, bin_size=10, window_size=100, format='binary'):
  """
  Spike train generator function.
  Builds trains over a given time window using discrete time bins. 

  Args:
    clustered_spikes : a dictionary of clusters
    start_time(ms)   : the starting time point of the window within a session
    bin_size(ms)     : the size of the discrete time step within the window
    window_size(ms)  : the width of the time window
    window_size(ms)  : binary or counts

  Returns:
    spike_trains     : the spikes trains of the neurons in the window
  """
  
  num_neurons = len(clustered_spikes.keys())
  num_bins = int(window_size/bin_size)

  if format == 'binary' or format == 'counts':
    # Initialize spike train matrix [num_neurons x num_bins]
    spike_trains = np.zeros((num_neurons,num_bins), dtype=int)

    # Iterate over all neurons in a recording session
    for i,cluster_id in enumerate(sorted(clustered_spikes.keys())):
      # Iterate over time bins in the window
      for j in range(num_bins):
        # Define bin start and end times
        bin_start = start_time + (j * bin_size)
        bin_end = start_time + ((j+1) * bin_size)
        
        # Convert spike times of a given neuron to millisecond scale
        neuron_spikes = clustered_spikes[cluster_id] * 1000 
        
        # Check if a spike occured in time bin [start, end)
        spikes = np.logical_and(neuron_spikes>=bin_start, neuron_spikes<bin_end)
        if format == 'binary':
          if True in spikes:
            spike_trains[i][j] = 1
        else:
          spike_trains[i][j] = np.count_nonzero(spikes==True) 
  else: # to be implemented
    pass
    
  return spike_trains


def clusters_to_area(cluster_ids, cluster_locs):
  # Get set of brain areas recorded
  brain_areas = set(cluster_locs.tolist())

  # Build output dict-> key=brain area, val=all clusters in brain area
  clusters_per_area = {area: [] for area in brain_areas}

  # Group clusters by brain area
  for id in cluster_ids:
    cluster_area = cluster_locs[id]
    clusters_per_area[cluster_area].append(id)
  
  # Convert lists to np arrays for efficiency 
  for area in clusters_per_area.keys():
    clusters_per_area[area] = np.array(sorted(clusters_per_area[area]))

  return clusters_per_area


def get_area_spike_trains(clustered_spikes, clusters_locs, start_time=0, bin_size=10, window_size=100, format='counts', aggregate=True, print_areas=False):
  num_bins = int(window_size/bin_size)
  clusters_by_area = clusters_to_area(clustered_spikes.keys(), clusters_locs)
  area_spikes = {area: np.zeros((clusters_by_area[area].shape[0], num_bins)) for area in clusters_by_area.keys()}

  # Process 
  for area in area_spikes.keys():
    if print_areas:
      print(f'Processing area: {area} - {len(clusters_by_area[area])} clusters')

    # Iterate through the clusters in an area
    for i,cluster_id in enumerate(clusters_by_area[area]):
      
      # Convert spike times of a given neuron to millisecond scale
      neuron_spikes = clustered_spikes[cluster_id] * 1000

      # Iterate through time bins
      for j in range(num_bins):
        
        # Define bin start and end times
        bin_start = start_time + (j * bin_size)
        bin_end = start_time + ((j+1) * bin_size)

        # Check if a spike occured in time bin [start, end)
        spikes = np.logical_and(neuron_spikes>=bin_start, neuron_spikes<bin_end)
      
        if format == 'binary':
          if True in spikes:
            area_spikes[area][i][j] = 1
        else: # Counts
          area_spikes[area][i][j] = np.count_nonzero(spikes==True)

    # Combines all area neuron behavior into a single array
    if aggregate==True and format=='counts':
      area_spikes[area] = area_spikes[area].sum(axis=0)
    elif aggregate==True and format=='binary':
      counts = area_spikes[area].sum(axis=0)
      area_spikes[area] = 1 * np.logical_and(counts > 0)

  return area_spikes

def group_trial_timestamps(intervals, vis_stims, go_cues, responses, feedback):
  num_trials = intervals.shape[0]
  trial_timepoints = np.zeros((num_trials, 6))

  for trial in range(num_trials):     
    # Trial start
    trial_timepoints[trial][0] = intervals[trial][0]
    # Visual stimulus
    trial_timepoints[trial][1] = vis_stims[trial][0]
    # Go gue
    trial_timepoints[trial][2] = go_cues[trial][0]
    # Response(wheel turn)
    trial_timepoints[trial][3] = responses[trial][0]
    # Feeback
    trial_timepoints[trial][4] = feedback[trial][0]
    # Trial end
    trial_timepoints[trial][5] = intervals[trial][1]

  return trial_timepoints


In [ ]:
# @title Load Data
#@markdown - Uses the path defined in BASE_PATH
#@markdown - Assumes each recording session resides in a separate subdirectory formatted as mouseID_year-month-date
#@markdown - **'mouse_id'**: (str) Mouse name per session directory format
#@markdown - **'session_date'**: (date) Date of session per session directory format
#@markdown - **'clustered_spikes'**: A dictionary of clusters(neurons) recorded during the session. Clusters with annotation values < 1 are not included.
#@markdown  - keys(int): The cluster's integer ID as specified in the datafiles.
#@markdown  - values(np.array): A 1-D array of size *nSpikes* where each entry corresponds to a time point in seconds during the recording session in which the cluster(neuron) produced a spike.
#@markdown - **'clusters_locs'**: (np.array) A 1-D  array of size *nClusters* where each entry corresponds to the the Allen CCF brain area of cluster *n*.
#@markdown  - This array includes **all** clusters from a recording session. Thus the keys of `clustered_spikes` should be used as the index number when accessing the values here.
#@markdown - **'trials_fb_times'**: trials.feedback_times.npy
#@markdown - **'trials_fb_type'**: trials.feedbackType.npy
#@markdown - **'trials_go_times'**: trials.goCue_times.npy
#@markdown - **'trials_included'**: trials.included.npy
#@markdown - **'trials_intervals'**: trials.intervals.npy
#@markdown - **'trials_rep_num'**: trials.repNum.npy
#@markdown - **'trials_resp_choice'**: trials.response_choice.npy
#@markdown - **'trials_resp_times'**: trials.response_times.npy
#@markdown - **'trials_vis_times'**: trials.visualStim_times.npy


all_session_data = dict()

# Order sessions by surname + date
for i,session in enumerate(tqdm(sorted(os.listdir(BASE_PATH)))):
  session_path = os.path.join(BASE_PATH, session)

  #print(f"Loading session: {session} ")

  # Load spike and cluster data
  raw_spikes = np.load(session_path+'/spikes.times.npy')
  raw_clusters = np.load(session_path+'/spikes.clusters.npy')
  cluster_annotations = np.load(session_path+'/clusters._phy_annotation.npy'), #cluster quality
  num_clusters = raw_clusters.max() 

  # Initialize 2-D list
  sorted_spike_times = [[] for i in range(num_clusters+1)]

  # Group spike times by their predicted cluster number 
  for j in range(len(raw_spikes)):
    sorted_spike_times[raw_clusters[j][0]].append(raw_spikes[j][0])
  
  # Dict to store valid clusters (annotation quality > 1)
  filtered_clusters = dict()

  # Convert clustered spikes to numpy arrays for efficiency
  for j,cluster in enumerate(sorted_spike_times):
    # Only store valid clusters
    if cluster_annotations[0][j][0] > 1:
      filtered_clusters[j] = np.array(sorted_spike_times[j])

  cluster_locs = []
  cluster_peak_channels = np.load(session_path+'/clusters.peakChannel.npy')
  brain_locs = pd.read_csv(session_path+'/channels.brainLocation.tsv', sep='\t')

  for cluster in cluster_peak_channels:
    # Offset area index by 1 due to 0-indexing in Pandas
    # clusters.peakChannel.npy appears to use 1-indexing
    idx = cluster[0]-1
    cluster_locs.append(brain_locs['allen_ontology'][idx])
  
  # Convert cluster brain locations to np array for efficiency
  cluster_locs = np.array(cluster_locs)

  # Load session data into dictionary
  all_session_data[i] = {
    # Session information
    'mouse_id': session.split('_')[0],
    'session_date': datetime.strptime(session.split('_')[1], '%Y-%m-%d').date(),

    # Neural Data 
    'clustered_spikes': filtered_clusters, # Dict: key=cluster_ID, val=spike_times(seconds)

    # Cluster Data
    'clusters_locs': cluster_locs,

    # Trial Data
    'trials_fb_type': np.load(session_path+'/trials.feedbackType.npy'),
    'trials_fb_times': np.load(session_path+'/trials.feedback_times.npy'),
    'trials_go_times': np.load(session_path+'/trials.goCue_times.npy'),
    'trials_included': np.load(session_path+'/trials.included.npy'),
    'trials_rep_num': np.load(session_path+'/trials.repNum.npy'),
    'trials_resp_choice': np.load(session_path+'/trials.response_choice.npy'),
    'trials_resp_times': np.load(session_path+'/trials.response_times.npy'),
    'trials_intervals': np.load(session_path+'/trials.intervals.npy'),
    'trials_vis_times': np.load(session_path+'/trials.visualStim_times.npy')
  }

In [ ]:
# @title File writer for Kobayashi method

#num_clusters = len(sorted_spike_times.keys())

with open("/content/drive/MyDrive/steinmetz/Cori_2016-12-14/kobayashi_datafile_5n.txt", "w") as f:
  for i,cluster in enumerate(sorted_spike_times.keys()):
    written = False
    
    # check if cluster is valid 
    if spike_clusters_annotations[cluster][0] < 2:
      continue

    for time in sorted_spike_times[cluster]:
      # limit spikes to first 5 minutes
      if time <= 10:
        written = True
        f.write(str(time)+'\n')

    #if i != num_clusters:
    # append only if a spike time has been written for a given neuron
    if written:    
      f.write(';\n')
  
  f.close()

### Example Helper Function calls

In [26]:
clustered_locs = clusters_to_area(all_session_data[0]['clustered_spikes'].keys(), all_session_data[0]['clusters_locs'])

In [28]:
area_spikes = get_area_spike_trains(all_session_data[0]['clustered_spikes'], all_session_data[0]['clusters_locs'], start_time=0, bin_size=10, window_size=100, format='counts', aggregate=True)

In [6]:
trials_times = group_trial_timestamps(all_session_data[0]['trials_intervals'], all_session_data[0]['trials_vis_times'], all_session_data[0]['trials_go_times'], all_session_data[0]['trials_resp_times'], all_session_data[0]['trials_fb_times'])

### Examining v2r and c2r statistics

In [ ]:
choice_code_areas = ['MOs', 'PL', 'MOp', 'CP', 'SNr', 'SCm', 'MRN', 'ZI']

In [16]:
delta_v2r = []
delta_c2r = []
engaged_trials = 0
total_trials = 0

for session in all_session_data.keys():
  grouped_trials_times = group_trial_timestamps(all_session_data[session]['trials_intervals'], all_session_data[session]['trials_vis_times'], all_session_data[session]['trials_go_times'], all_session_data[session]['trials_resp_times'], all_session_data[session]['trials_fb_times'])
  session_areas = set(all_session_data[session]['clusters_locs'])

  aoi_present = False
  for area in choice_code_areas:
    if area in session_areas:
      aoi_present = True
      break

  if aoi_present == True:
    #print(f'SESSION: {session} - choice areas detected')
    
    session_engaged_trials = 0
    for i,trial in enumerate(grouped_trials_times):
      total_trials += 1
      if all_session_data[session]['trials_included'][i] == True:
        engaged_trials += 1
        session_engaged_trials += 1
        delta_v2r.append(trial[3] - trial[1]) # vis stim to response delta
        delta_c2r.append(trial[3] - trial[2]) # cue to response delta
    #print(f'Engaged trials: {session_engaged_trials}/{len(grouped_trials_times)}')
  else:
    #print(f'SESSION: {session} - choice areas not recorded')
    continue

In [ ]:
delta_v2r = np.array(delta_v2r)
delta_c2r = np.array(delta_c2r)

avg_v2r = np.average(delta_v2r)
med_v2r = np.median(delta_v2r)
max_v2r = delta_v2r.max()
min_v2r = delta_v2r.min()

avg_c2r = np.average(delta_c2r)
med_c2r = np.median(delta_c2r)
max_c2r = delta_c2r.max()
min_c2r = delta_c2r.min()

print(f'V2R:\n  avg: {avg_v2r}\n  med: {med_v2r}\n  max: {max_v2r}\n  min: {min_v2r}')
print(f'C2R:\n  avg: {avg_c2r}\n  med: {med_c2r}\n  max: {max_c2r}\n  min: {min_c2r}')

In [ ]:
data = [delta_v2r, delta_c2r]
fig = plt.figure(figsize =(10, 7))

# Creating axes instance
ax = fig.add_axes([0, 0, 1, 1])

# Creating plot
plt.boxplot(data)

# x-axis labels
ax.set_xticklabels(['v2r', 'c2r'])

plt.xlabel('Deltas')
plt.ylabel('Seconds')

# Removing top axes and right axes
# ticks
ax.get_xaxis().tick_bottom()
ax.get_yaxis().tick_left()

# show plot
plt.show()

## Predicting Mouse Action Choice 

In [10]:
BIN_SIZE = 5 # milliseconds
WINDOW_SIZE = 100 # milliseconds
choice_code_areas = ['MOs', 'PL', 'MOp', 'CP', 'SNr', 'SCm', 'MRN', 'ZI']

In [ ]:
#@title Process Steinmetz Data
#@markdown We iterate through all recording sessions and build a dataset of binned spike counts for our areas of interest using a selected window and bin size.
#@markdown - Per the Steinmetz results, choice selective behavior occurs 50-100ms before movement onset.
neural_data = []
choice_data = []

for session in all_session_data.keys():
  session_areas = set(all_session_data[session]['clusters_locs'])
  mouse_id = all_session_data[session]['mouse_id']
  session_date = all_session_data[session]['session_date']

  aoi_present = False
  # Check if at least one choice area was recorded in session
  for area in choice_code_areas:
    if area in session_areas:
      aoi_present = True
      break

  # Ignore sessions where choice areas were not recorded
  if aoi_present == True:
    print(f'SESSION: {session}_{mouse_id}_{session_date} >> choice areas detected')
    grouped_trials_times = group_trial_timestamps(all_session_data[session]['trials_intervals'], all_session_data[session]['trials_vis_times'], all_session_data[session]['trials_go_times'], all_session_data[session]['trials_resp_times'], all_session_data[session]['trials_fb_times'])

    # Iterate through session trials
    for i,trial in enumerate(tqdm(grouped_trials_times, desc='Session Trials')):
      
      # Stop after n trials (sanity check if we are using the print statement below)
      #if i > 3:
      #  break

      # Only include trials where mouse was engaged
      if all_session_data[session]['trials_included'][i] == True:
        # Response timestamp - window size
        start = int(trial[3]*1000 - WINDOW_SIZE)

        # Get neural data in window of interest
        area_spikes = get_area_spike_trains(all_session_data[session]['clustered_spikes'], all_session_data[session]['clusters_locs'], start_time=start, bin_size=BIN_SIZE, window_size=WINDOW_SIZE, aggregate=True)
        
        # Check for neural data of each choice area
        for area in choice_code_areas:
          if area in session_areas:
            neural_data.append(area_spikes[area])
            choice_data.append(all_session_data[session]['trials_resp_choice'][i]) # Right: -1, Left: +1, NoGo: 0
            #print(session, i, area, area_spikes[area], all_session_data[session]['trials_resp_choice'][i])
  else:
    print(f'SESSION: {session}_{mouse_id}_{session_date} >> choice areas not recorded')

neural_data = np.array(neural_data)
choice_data = np.array(choice_data)

In [23]:
# Save data arrays (optional)
np.save(BASE_PATH+'/neural_data.npy', neural_data)
np.save(BASE_PATH+'/choice_data.npy', choice_data)

In [27]:
#@title Define SpikeDataset Class
#@markdown We create a custom Pytorch dataset class to handle the neural and choice data, which allows the use of Pytorch's Dataloader class during training and evaluation.

class SpikeDataset(Dataset):
  def __init__(self, spike_matrix, spike_labels, transform=None, target_transform=None):
    self.spike_data = torch.from_numpy(spike_matrix)#.float()
    self.spike_labels = torch.from_numpy(spike_labels)#.float()#.long()
    self.transform = transform
    self.target_transform = target_transform

  def __len__(self):
    return len(self.spike_labels)

  def __getitem__(self, idx):
    spike_train = self.spike_data[idx]
    label = self.spike_labels[idx]
    if self.transform:
        pass
    if self.target_transform:
        pass
    return spike_train, label

In [24]:
#@title Define Decoding Model

class NeuralDecoderNetwork(nn.Module):
    def __init__(self, input_shape, output_shape):
        super(NeuralDecoderNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_shape, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.BatchNorm1d(32),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.BatchNorm1d(16),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.BatchNorm1d(8),
            #nn.Dropout(p=0.1),
            nn.Linear(8, 1),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [19]:
#@title Define Train and Evaluation Loops

def train_loop(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)

  model.train()
  for batch, (X, y) in enumerate(dataloader):
    # Compute prediction and loss
    X, y = X.to(device), y.to(device)

    pred = model(X)
    loss = loss_fn(pred, y.unsqueeze(1))

    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 10 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  test_loss, correct = 0, 0

  model.eval()
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)

      pred = model(X)
      test_loss += loss_fn(pred, y.unsqueeze(1)).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()

  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
# Load neural and choice data from files (optional)
neural_data = np.load(BASE_PATH+'/cleaned_neural_data.npy')
choice_data = np.load(BASE_PATH+'/cleaned_choice_data.npy')

### Train Model

In [30]:
BATCH_SIZE = 64
LR = 0.01 #1e-3
EPOCHS = 50

In [28]:
X_train, X_test, y_train, y_test = train_test_split(neural_data, choice_data, test_size=0.2)

training_data = SpikeDataset(X_train, y_train)
test_data = SpikeDataset(X_test, y_test)

train_dataloader = DataLoader(training_data, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
input_shape = neural_data.shape[1]
num_classes = len(np.unique(choice_data)) if len(np.unique(choice_data)) > 2 else 1

# Initialize model
model = NeuralDecoderNetwork(input_shape, num_classes).to(device)
print(model)

In [32]:
# Initialize the loss function & optimizer
#loss_fn = nn.CrossEntropyLoss()
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

In [ ]:
for t in range(EPOCHS):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")